<a href="https://colab.research.google.com/github/hamednasr/TensorFlow-Projects/blob/main/Analyzing_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Tweet`s overal message(disaster or not?):
## This is a kaggle competition.
https://www.kaggle.com/competitions/nlp-getting-started/data

In [2]:
import tensorflow as tf
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-08-29 18:50:01--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-29 18:50:01 (77.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
from helper_functions import unzip_data, plot_loss_curves, compare_historys

In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-08-29 18:50:01--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.128, 172.217.204.128, 172.217.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-08-29 18:50:02 (92.4 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [6]:
unzip_data('/content/nlp_getting_started.zip')

## Data Exploration:

In [7]:
train_data = pd.read_csv('train.csv')
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
train_data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
test_data = pd.read_csv('test.csv')
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [10]:
random_data = train_data.sample(frac=0.0006)
for i in range(len(random_data)):
  print(f'tweet number {i+1} :')
  if random_data.iloc[i, 4]==1:
    target = 'a Disaster'
  else:
    target = 'NOT Disaster'
  print(f'{random_data.iloc[i, 3]} : This is {target} tweet')
  print('------------------------------------\n')


tweet number 1 :
#Obama signed up to a deal that far from making the world a safer place http://t.co/E0luGBL6pb via @upi #Iran #Nuclear #IranNuclearDeal : This is a Disaster tweet
------------------------------------

tweet number 2 :
Philadelphia EaglesÛª Jordan Matthews Is Going To Explode In 2015 http://t.co/rRq1ildkiL #news #hotnewscake : This is NOT Disaster tweet
------------------------------------

tweet number 3 :
From recycling to only using non-hazardous chemicals Holland 1916 continually strives to maintain an eco-friendly existence. : This is a Disaster tweet
------------------------------------

tweet number 4 :
You can't watch 'Home Alone 2' without telling your kids 
'she used to be on Casualty'. 
#Homealone2 #film4 : This is NOT Disaster tweet
------------------------------------

tweet number 5 :
@time4me_sews I know! It still hasn't quite sunk in :D : This is NOT Disaster tweet
------------------------------------



## Extract validation data from train data:

In [11]:
train_data['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [12]:
train_data['target'].to_numpy()

array([1, 1, 1, ..., 1, 1, 1])

In [13]:
train_data, val_data, train_label, val_label = train_test_split(train_data['text'].to_numpy(), train_data['target'].to_numpy(), 
                                                                test_size=0.15, 
                                                                random_state=42)

In [14]:
val_data[:10], val_label[:10]

(array(['So you have a new weapon that can cause un-imaginable destruction.',
        'The f$&amp;@ing things I do for #GISHWHES Just got soaked in a deluge going for pads and tampons. Thx @mishacollins @/@',
        'DT @georgegalloway: RT @Galloway4Mayor: \x89ÛÏThe CoL police can catch a pickpocket in Liverpool Stree... http://t.co/vXIn1gOq4Q',
        'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.',
        'in response to trauma Children of Addicts develop a defensive self - one that decreases vulnerability. (3',
        '@Calum5SOS you look like you got caught in a rainstorm this is amazing and disgusting at the same time',
        'my favorite lady came to our volunteer meeting\nhopefully joining her youth collision and i am excite http://t.co/Ij0wQ490cS',
        '@brianroemmele UX fail of EMV - people want to insert and remove quickly like a gas pump stripe reader. 1 person told me it crashed the POS',
       

## Tokenization (Vectorization):

In [15]:
sum([len(tweet.split()) for tweet in train_data])

96401

In [16]:
round(sum([len(tweet.split()) for tweet in train_data])/len(train_data))

15

In [17]:
text_vectorization = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                       output_mode='int', 
                                                       output_sequence_length=15)

In [18]:
text_vectorization.adapt(train_data)

In [19]:
sample_tweet = 'my name is hamed i am happy to meet you!'
text_vectorization(sample_tweet)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([  13,  736,    9,    1,    8,  161,  681,    5, 1732,   12,    0,
          0,    0,    0,    0])>

In [20]:
random_tweet = random.choice(train_data)
print('tweet: ', random_tweet,'\n\n')
print('vectorized tweet: ', text_vectorization(random_tweet))

tweet:  RT patrickjbutler: Excellent damiengayle eyewitness account of Kids Company closure: 'You drop the bomb and expectÛ_ http://t.co/pHH1VmLfoo 


vectorized tweet:  tf.Tensor(
[ 110 9540 3713    1  562 1270    6  561 1495    1   12 1781    2  115
    7], shape=(15,), dtype=int64)


In [21]:
#unique words
unique_words = text_vectorization.get_vocabulary()
print(len(unique_words))
print(unique_words[:10])
print(unique_words[-10:])

10000
['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']
['nkulw', 'njenga', 'nj36', 'nittys', 'nitroglycerin', 'nitishkumar', 'nite', 'nissannews', 'nissan', 'niqqa']


## Embedding:

In [31]:
embedding = tf.keras.layers.Embedding(input_dim = 10000, output_dim = 32, input_length=None)

In [32]:
sample_embed = embedding(text_vectorization(random_tweet))
sample_embed

<tf.Tensor: shape=(15, 32), dtype=float32, numpy=
array([[ 4.85596322e-02,  1.99453942e-02,  4.84256260e-02,
         4.07626368e-02, -2.93894056e-02, -3.78947258e-02,
         3.26809548e-02,  3.02758254e-02,  1.29705109e-02,
        -1.30880363e-02, -3.98523733e-03,  4.31891792e-02,
         1.41895451e-02, -2.39767879e-03, -2.93552764e-02,
        -3.36239338e-02,  3.34553160e-02, -1.12325773e-02,
        -1.85051337e-02,  1.63529404e-02, -2.41502374e-03,
        -8.49064440e-03, -4.06452194e-02,  1.83264948e-02,
        -4.08911929e-02,  2.82916315e-02,  8.42920691e-03,
        -2.45967396e-02,  1.61089562e-02, -1.33366100e-02,
        -1.87038071e-02,  1.21319890e-02],
       [ 3.82091664e-02, -4.15164232e-03, -1.49996392e-02,
        -1.57485120e-02,  8.54682177e-04,  1.98920481e-02,
         4.78547849e-02, -4.64179404e-02, -1.34839416e-02,
         2.32947730e-02, -2.53740698e-03,  3.70474122e-02,
         1.73270702e-04, -2.32557412e-02,  4.12638672e-02,
        -3.85971777e-0

## Analyze with Naive Bayes using TF-IDF:

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

In [34]:
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('clf', MultinomialNB())
])

model_0.fit(train_data,train_label)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [35]:
val_pred = model_0.predict(val_data)
val_pred

array([0, 0, 0, ..., 1, 0, 0])

In [36]:
val_label

array([1, 0, 1, ..., 1, 0, 0])

In [37]:
accuracy_score(val_label, val_pred)

0.7863397548161121

In [38]:
print(classification_report(val_label, val_pred))

              precision    recall  f1-score   support

           0       0.76      0.92      0.83       651
           1       0.85      0.61      0.71       491

    accuracy                           0.79      1142
   macro avg       0.80      0.76      0.77      1142
weighted avg       0.80      0.79      0.78      1142



## Analyze with Vanilla Neural Network:

In [ ]:
from helper_functions import create_tensorboard_callback
model_1 = create_tensorboard_callback('')

In [39]:
 model_1 = tf.keras.Sequential([
                      tf.keras.layers.Dense(64,input_shape=(32), activation='relu'),
                      tf.keras.layers.Dense(128, activation='relu'),
                      tf.keras.layers.Dense(32, activation='relu'),
                      tf.keras.layers.Dense(1, activation='sigmoid')
 ])

 model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])
 
 model_1.build()
 model_1.summary()